In [1]:
import pandas as pd
import numpy as np

TWEETS_FILE_PATH='C:/Users/zaada/Documents/school/Intro to Data Science/Final_task/Part 2/data/tweets_metrics.csv'
USER_FILE_PATH='C:/Users/zaada/Documents/school/Intro to Data Science/Final_task/Part 2/data/users_metrics.csv'

tweets = pd.read_csv(TWEETS_FILE_PATH,encoding='latin-1').rename({'created at': 'created_at'}, axis=1)
users = pd.read_csv(USER_FILE_PATH,encoding='latin-1')


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/zaada/Documents/school/Intro to Data Science/Final_task/Part 2/data/tweets_metrics.csv'

In [ ]:
tweets["shared_geo_location"] = tweets.apply(lambda row: 0 if (type(row.geo) is float) else 1, axis = 1)
tweets["hashtag_count"] = tweets.apply(lambda row: row.hashtags.count('text'), axis = 1)
tweets["device"] = tweets.apply(lambda row: 'PC' if ('Twitter for Websites' in row.source or 'Twitter Web Client' in row.source) else 'mobile', axis = 1)
tweets["word_count"] = tweets.apply(lambda row: len(row.text.split()), axis = 1)
tweets


In [ ]:
users['Num_words_in_desc'] = users.apply(lambda row: 0 if(type(row.description) is float) else len(row.description.split()), axis = 1)
users['Is_celeb'] = users.apply(lambda row: 1 if(int(row.followers_count) > 100000) else 0, axis = 1)
ids = users.user_id
Collected_tweets = []
for Id in ids:
    Collected_tweets.append(len(tweets.loc[tweets.user_id == Id]))
users['Collected_tweets'] = Collected_tweets
users['Collected_tweets_percent'] = users.apply(lambda row: row.Collected_tweets / row.statuses_count, axis = 1)
users

In [ ]:
import datetime
temptweets = tweets.copy(deep=True)
temptweets.dropna(subset = ['created_at'], inplace=True)## drop what is null#
temptweets['created_at']=temptweets['created_at'].astype('datetime64[ns]').apply(lambda row: row.strftime("%x")).astype('datetime64[ns]')# cast it into datetime  do it in m/d/y ##
temptweets['year']= temptweets['created_at'].apply(lambda row: row.strftime("%Y"))#get the year##
temptweets['month']= temptweets['created_at'].apply(lambda row: row.strftime("%b"))#get the mounth ##
temptweets['mobile']= temptweets['device'].apply(lambda row: 1 if (row=="mobile" ) else 0)#the device will be 1 if mobile will use it later ##
# itay zaada is the king of the all world ##
temptweets['quoted']= temptweets['is_quote_status'].apply(lambda row: 1 if (row=="True" ) else 0)# if it was quted it will be 1#

#all of this dataframe is temp ##

In [ ]:
tweets_mothly_summary = temptweets.groupby(['user_id','year','month']).agg(
{
"tweet_id": "count",
"hashtag_count":"sum",
"mobile":"sum",
'retweet_count': 'sum',
'shared_geo_location':'sum',
'quoted' :'sum',
    }).reset_index() ##agg according id year and month ##


tweets_mothly_summary.columns = ['user_id','Year','Month','Tweet_count',' Hashtag_count',"mobile","Retweet_count","Location_sharing_count","Quote_count"]
temp2=users.copy(deep=True)[['user_id','statuses_count']]                         

tweets_mothly_summary=pd.merge(tweets_mothly_summary, temp2,on='user_id',how='left')## insert the total num in the dataframe ##
tweets_mothly_summary['Percent_mobile'] = tweets_mothly_summary['mobile']/tweets_mothly_summary['Tweet_count']#make a mobile precent # 

tweets_mothly_summary['from total'] = tweets_mothly_summary['Tweet_count']/tweets_mothly_summary['statuses_count']
tweets_mothly_summary = tweets_mothly_summary.drop(['mobile'], axis=1)
tweets_mothly_summary = tweets_mothly_summary.drop(['statuses_count'], axis=1)

In [ ]:
tweets_mothly_summary


In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=80)
##hist of tweets for person ##
x=users['statuses_count']
histo=x.hist(bins=20,color='pink')
histo.set_ylabel('number of people')
histo.set_xlabel('number of tweets')
plt.show()

In [ ]:
## hist for followers per person ##
from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=80)
y=users['followers_count']
histo2=y.hist(bins=20,color='gold')
histo2.set_ylabel('number of people')
histo2.set_xlabel('number of followers')
histo2.ticklabel_format(useOffset=False, style='plain')
plt.show()


In [ ]:
from matplotlib.pyplot import figure
df1 = users[['user_id','followers_count']].sort_values('followers_count',ascending = False).head(10)

plot_bar=df1.plot.bar(x="user_id", y="followers_count", rot=80, title="Number of followers for the top 10",color=['brown', 'red', 'green', 'blue', 'cyan',"yellow","pink","magenta","orange","gray"])
plt.show()

In [ ]:
from matplotlib.pyplot import figure
df2 = users[['statuses_count','followers_count']].sort_values('followers_count',ascending = False)

plot_line=df2.plot(x="followers_count", y="statuses_count", rot=70, title="followers_count vs statuses_count",c='brown',linewidth=3.5)
plot_line.ticklabel_format(useOffset=False, style='plain')
plot_line.grid() 



plt.show()

In [ ]:
tweets_mothly_summary.to_csv('tweets_mothly_summary.csv',index=False ,encoding="utf-8-sig") ## make it in csv ##
users.to_csv('users.csv',index=False ,encoding="utf-8-sig") ## make it in csv ##
tweets.to_csv('tweets.csv',index=False ,encoding="utf-8-sig") ## make it in csv ##
 